# This Class is used to handle the images the user uploads

The images will be cropped and rotated to prepare for embedding and querying the vector database.

I have not been successful in this endeavor so far.


In [5]:
import cv2
import numpy as np

def detect_and_crop_outer_card(image_path, min_area=5000):
    """
    Detect and crop the outer rectangle (the entire card) from the image.
    Automatically rotate the card to be vertical without altering colors.
    
    Args:
    - image_path: Path to the image containing the cards.
    - min_area: Minimum area to consider a contour as a card.
    
    Returns:
    - cropped_card: Cropped and rotated card image.
    """
    # Load the image without any color alterations
    image = cv2.imread(image_path)
    
    # Convert to grayscale for edge detection (no color alteration)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    
    # Edge detection to find card contours
    edges = cv2.Canny(blurred, 50, 150)
    
    # Find contours
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Find the largest contour which likely represents the card
    largest_contour = None
    max_area = 0
    
    for contour in contours:
        area = cv2.contourArea(contour)
        if area > max_area:
            max_area = area
            largest_contour = contour
    
    # If no valid contour was found, return None
    if largest_contour is None or max_area < min_area:
        print("No card detected or card is too small.")
        return None
    
    # Get the minimum area rectangle for the largest contour (the outer card)
    rect = cv2.minAreaRect(largest_contour)
    box = cv2.boxPoints(rect).astype(np.intp)  # Use np.intp instead of np.int0
    
    # Crop the card from the image using the bounding box
    width = int(rect[1][0])
    height = int(rect[1][1])
    
    # Get the rotation matrix
    src_pts = box.astype("float32")
    dst_pts = np.array([[0, height-1],
                        [0, 0],
                        [width-1, 0],
                        [width-1, height-1]], dtype="float32")
    
    M = cv2.getPerspectiveTransform(src_pts, dst_pts)
    warped = cv2.warpPerspective(image, M, (width, height))
    
    # Ensure the card is rotated to be vertical
    if height > width:
        warped = cv2.rotate(warped, cv2.ROTATE_90_CLOCKWISE)
    
    return warped


In [10]:
import cv2
import numpy as np

def detect_and_crop_cards(image_path, min_area=5000):
    """
    Detect and crop cards from the image.
    Automatically rotate them to be vertical based on their dimensions.
    
    Args:
    - image_path: Path to the image containing the cards.
    - min_area: Minimum area to consider a contour as a card.
    
    Returns:
    - cropped_cards: List of cropped and rotated card images.
    """
    # Load the image without any color alterations
    image = cv2.imread(image_path)
    
    # Convert to grayscale for edge detection (no color alteration)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Use adaptive thresholding to better handle complex backgrounds
    thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 11, 2)
    
    # Apply dilation to close gaps in the edges (e.g., caused by rounded corners or noise)
    kernel = np.ones((5, 5), np.uint8)
    dilated = cv2.dilate(thresh, kernel, iterations=2)
    
    # Find contours
    contours, _ = cv2.findContours(dilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    cropped_cards = []
    
    for contour in contours:
        area = cv2.contourArea(contour)
        if area < min_area:
            continue  # Skip small contours
        
        # Get the bounding rectangle for the contour
        x, y, w, h = cv2.boundingRect(contour)
        
        # Ensure the card is rotated to be vertical (based on dimensions)
        if w > h:  # Card is in landscape orientation
            card = image[y:y+h, x:x+w]
            card = cv2.rotate(card, cv2.ROTATE_90_CLOCKWISE)
        else:  # Card is already in portrait orientation
            card = image[y:y+h, x:x+w]
        
        cropped_cards.append(card)
    
    return cropped_cards

image_paths = [
    'Datasets/mtg_test_images/shuko_mountain_carpet.jpg',
    'Datasets/mtg_test_images/shuko_single_held_in_hand.jpg', 
    'Datasets/mtg_test_images/shuko_single_rotated_countertop.jpg',
    'Datasets/mtg_test_images/shuko_single_slightly_rotated_countertop.jpg', 
    'Datasets/mtg_test_images/shuko_single_slightly_rotated_tablecloth.jpg', 
    'Datasets/mtg_test_images/shuko.jpg'
]

# Detect and crop the outer card from each image
for image_idx, image_path in enumerate(image_paths):
    cards = detect_and_crop_cards(image_path)
    
    # Save each detected card
    for card_idx, card in enumerate(cards):
        output_filename = f"cropped_outer_card_{image_idx}_{card_idx}.jpg"
        cv2.imwrite(output_filename, card)
        print(f"Saved: {output_filename}")


Saved: cropped_outer_card_0_0.jpg
Saved: cropped_outer_card_1_0.jpg
Saved: cropped_outer_card_2_0.jpg
Saved: cropped_outer_card_3_0.jpg
Saved: cropped_outer_card_4_0.jpg
Saved: cropped_outer_card_5_0.jpg


In [25]:
import cv2
import numpy as np
from shapely.geometry import Polygon
import os

class CardDetector:
    def __init__(self):
        pass

    def order_polygon_points(self, x, y):
        """
        Orders polygon points into a counterclockwise order.
        """
        angle = np.arctan2(y - np.average(y), x - np.average(x))
        ind = np.argsort(angle)
        return (x[ind], y[ind])

    def four_point_transform(self, image, poly):
        """
        A perspective transform for a quadrilateral polygon.
        """
        pts = np.zeros((4, 2))
        pts[:, 0] = np.asarray(poly.exterior.coords)[:-1, 0]
        pts[:, 1] = np.asarray(poly.exterior.coords)[:-1, 1]
        
        # Obtain a consistent order of the points and unpack them
        rect = np.zeros((4, 2))
        (rect[:, 0], rect[:, 1]) = self.order_polygon_points(pts[:, 0], pts[:, 1])

        # Compute the width of the new image
        width_a = np.sqrt(((rect[1, 0] - rect[0, 0]) ** 2) +
                          ((rect[1, 1] - rect[0, 1]) ** 2))
        width_b = np.sqrt(((rect[3, 0] - rect[2, 0]) ** 2) +
                          ((rect[3, 1] - rect[2, 1]) ** 2))
        max_width = max(int(width_a), int(width_b))

        # Compute the height of the new image
        height_a = np.sqrt(((rect[0, 0] - rect[3, 0]) ** 2) +
                           ((rect[0, 1] - rect[3, 1]) ** 2))
        height_b = np.sqrt(((rect[1, 0] - rect[2, 0]) ** 2) +
                           ((rect[1, 1] - rect[2, 1]) ** 2))
        max_height = max(int(height_a), int(height_b))

        # Prepare the transform
        rect = np.array(rect, dtype="float32")
        dst = np.array([
            [0, 0],
            [max_width - 1, 0],
            [max_width - 1, max_height - 1],
            [0, max_height - 1]], dtype="float32")

        # Compute the perspective transform matrix and then apply it
        transform = cv2.getPerspectiveTransform(rect, dst)
        warped = cv2.warpPerspective(image, transform, (max_width, max_height))

        return warped

    def convex_hull_polygon(self, contour):
        """
        Returns the convex hull of the given contour as a polygon.
        """
        if len(contour) >= 4:  # Ensure the contour has at least 4 points
            hull = cv2.convexHull(contour)
            phull = Polygon([[x, y] for (x, y) in zip(hull[:, :, 0], hull[:, :, 1])])
            return phull
        else:
            return None  # Ignore contours with fewer than 4 points

    def get_largest_card_contour(self, contours):
        """
        Find the largest contour that is likely to represent a card.
        """
        largest_contour = None
        largest_area = 0

        for contour in contours:
            # Approximate contour to a polygon
            epsilon = 0.02 * cv2.arcLength(contour, True)
            approx = cv2.approxPolyDP(contour, epsilon, True)

            if len(approx) == 4:  # Filter to keep only quadrilaterals
                area = cv2.contourArea(approx)
                if area > largest_area:
                    largest_area = area
                    largest_contour = approx

        return largest_contour

    def contour_image_gray(self, image):
        """
        Grayscale transform, thresholding, and contouring.
        """
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        fltr_size = 1 + 2 * (min(image.shape[0], image.shape[1]) // 20)
        thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, fltr_size, 10)
        contours, _ = cv2.findContours(np.uint8(thresh), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        return contours

    def detect_and_crop_cards(self, image_path, output_folder):
        """
        Detects and crops the largest card-like contour from an image.
        """
        image = cv2.imread(image_path)
        contours = self.contour_image_gray(image)

        largest_contour = self.get_largest_card_contour(contours)
        if largest_contour is not None:
            hull_poly = self.convex_hull_polygon(largest_contour)
            if hull_poly is not None:  # Ensure we have a valid polygon
                warped = self.four_point_transform(image, hull_poly)

                # Save the result
                output_filename = os.path.join(output_folder, os.path.basename(image_path))
                cv2.imwrite(output_filename, warped)
                print(f"Saved: {output_filename}")


In [27]:
detector = CardDetector()
image_paths = [
    'Datasets/mtg_test_images/shuko_mountain_carpet.jpg',
    'Datasets/mtg_test_images/shuko_single_held_in_hand.jpg', 
    'Datasets/mtg_test_images/shuko_single_rotated_countertop.jpg',
    'Datasets/mtg_test_images/shuko_single_slightly_rotated_countertop.jpg', 
    'Datasets/mtg_test_images/shuko_single_slightly_rotated_tablecloth.jpg', 
    'Datasets/mtg_test_images/shuko.jpg'
]

output_folder = 'Datasets/test_processed_image_output/'

# Process each image
for image_path in image_paths:
    detector.detect_and_crop_cards(image_path, output_folder)


Saved: Datasets/test_processed_image_output/shuko_mountain_carpet.jpg
Saved: Datasets/test_processed_image_output/shuko_single_held_in_hand.jpg
Saved: Datasets/test_processed_image_output/shuko_single_rotated_countertop.jpg
Saved: Datasets/test_processed_image_output/shuko_single_slightly_rotated_countertop.jpg
Saved: Datasets/test_processed_image_output/shuko_single_slightly_rotated_tablecloth.jpg
Saved: Datasets/test_processed_image_output/shuko.jpg


# Binder page test

In [35]:
import cv2
import numpy as np
from shapely.geometry import Polygon
import os

class CardDetector:
    def __init__(self):
        self.aspect_ratio = 63 / 88  # Typical aspect ratio of a Magic card (63mm x 88mm)
    
    def preprocess_image(self, image):
        """Convert the image to grayscale and apply adaptive thresholding."""
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        blurred = cv2.GaussianBlur(gray, (5, 5), 0)
        thresh = cv2.adaptiveThreshold(blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 11, 2)
        return thresh
    
    def find_card_contours(self, thresh):
        """Find contours that could potentially be cards."""
        contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        card_contours = []
        
        for contour in contours:
            epsilon = 0.02 * cv2.arcLength(contour, True)
            approx = cv2.approxPolyDP(contour, epsilon, True)
            
            if len(approx) == 4:
                polygon = Polygon(approx.reshape(4, 2))
                min_x, min_y, max_x, max_y = polygon.bounds
                width = max_x - min_x
                height = max_y - min_y
                aspect_ratio = width / height
                
                # Filter by aspect ratio and area
                if 0.8 * self.aspect_ratio < aspect_ratio < 1.2 * self.aspect_ratio and width * height > 10000:
                    card_contours.append(approx)
        
        return card_contours
    
    def crop_and_transform_card(self, image, contour):
        """Crop and apply perspective transform to the detected card."""
        pts = contour.reshape(4, 2)
        rect = np.zeros((4, 2), dtype="float32")
        
        # Order the points to match the card's corners
        s = pts.sum(axis=1)
        rect[0] = pts[np.argmin(s)]  # Top-left
        rect[2] = pts[np.argmax(s)]  # Bottom-right
        
        diff = np.diff(pts, axis=1)
        rect[1] = pts[np.argmin(diff)]  # Top-right
        rect[3] = pts[np.argmax(diff)]  # Bottom-left
        
        # Dimensions of the new image
        width = max(np.linalg.norm(rect[2] - rect[3]), np.linalg.norm(rect[1] - rect[0]))
        height = max(np.linalg.norm(rect[1] - rect[2]), np.linalg.norm(rect[0] - rect[3]))
        
        dst = np.array([
            [0, 0],
            [width - 1, 0],
            [width - 1, height - 1],
            [0, height - 1]], dtype="float32")
        
        # Perspective transform
        M = cv2.getPerspectiveTransform(rect, dst)
        warped = cv2.warpPerspective(image, M, (int(width), int(height)))
        
        return warped
    
    def detect_and_crop_cards(self, image_path, output_folder):
        """Detect cards in an image and save the cropped images."""
        image = cv2.imread(image_path)
        thresh = self.preprocess_image(image)
        card_contours = self.find_card_contours(thresh)
        
        if not os.path.exists(output_folder):
            os.makedirs(output_folder)
        
        for idx, contour in enumerate(card_contours):
            cropped_card = self.crop_and_transform_card(image, contour)
            output_path = os.path.join(output_folder, f'card_{idx + 1}.jpg')
            cv2.imwrite(output_path, cropped_card)
            print(f"Saved {output_path}")


detector = BinderCardDetector()

# Image file and output folder
image_path = 'Datasets/mtg_test_images/binder_page.jpeg'
output_folder = 'Datasets/test_processed_image_output/'

# Detect and crop cards
detector.detect_and_crop_cards(image_path, output_folder)


Saved: Datasets/test_processed_image_output/card_1.jpg
Saved: Datasets/test_processed_image_output/card_2.jpg
Saved: Datasets/test_processed_image_output/card_3.jpg
Saved: Datasets/test_processed_image_output/card_4.jpg
